<a href="https://colab.research.google.com/github/paulokuriki/prompt_engineering/blob/main/prompt_engineering.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 📑 **Radiology Report Classification with Large Language Models**

## **Learning Objectives**

By the end of this session, you will be able to:


✅ Understand how Large Language Models (LLMs) assist in radiology report classification.  
✅ Apply effective prompting techniques for medical report analysis.  
✅ Develop specialized classifiers for different radiological findings.  

---

## **Introduction**
Radiologists frequently need to classify reports based on specific findings or conditions. **Large Language Models (LLMs)** can streamline this process, improving efficiency and consistency in medical report classification. This tutorial demonstrates how to leverage LLMs for various classification tasks in radiology.

---

## **Dataset: Indiana Chest X-ray Collection**
This notebook processes the **Indiana Chest X-ray Collection**, a publicly available dataset provided by the **National Library of Medicine (NLM), National Institutes of Health (NIH),** in collaboration with **Indiana University**.

### 🎯 **Acknowledgment**
- **Dataset Source**: [Open-i (NLM)](https://openi.nlm.nih.gov/)  
- **Reference Paper**:  
  > **Demner-Fushman D, Kohli MD, Rosenman MB, Shooshan SE, Rodriguez L, Antani S, Thoma GR, McDonald CJ.**  
  > *Preparing a collection of radiology examinations for distribution and retrieval.*  
  > J Am Med Inform Assoc. 2016 Mar;23(2):304-10.  
  > DOI: [10.1093/jamia/ocv080](https://doi.org/10.1093/jamia/ocv080)  
  > PMID: [26133894](https://pubmed.ncbi.nlm.nih.gov/26133894/) | PMCID: [PMC5009925](https://www.ncbi.nlm.nih.gov/pmc/articles/PMC5009925/)  

## **Setup and Data Loading**
To begin, we'll install the required libraries and load the dataset.

In [1]:
pip install langchain_openai langchain-ollama

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.9/54.9 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 35.5 MB/s eta 0:00:00


In [2]:
import os
import getpass
import random
import json
import requests
import tarfile
import glob
import csv
import shutil
import xml.etree.ElementTree as ET

import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns
from tqdm import tqdm
from langchain_openai import ChatOpenAI
from langchain_ollama import ChatOllama
from IPython.display import HTML, display

## **Dataset Processing**
### **Steps**
1. **Download**: Retrieve the dataset from the official NLM repository.  
2. **Extract**: Unpack the `.tgz` archive.  
3. **Parse**: Extract the **Findings** section from each radiology report.  
4. **Label**: Assign a **normal** or **abnormal** classification based on MeSH (Medical Subject Headings) terms.  
5. **Export**: Save the processed data as a **CSV file**.  

In [3]:
import os
import shutil
import requests
import tarfile
import glob
import xml.etree.ElementTree as ET
import csv
import pandas as pd
from tqdm import tqdm

# Define URLs and paths
TGZ_URL = "https://openi.nlm.nih.gov/imgs/collections/NLMCXR_reports.tgz"
TGZ_FILE = "NLMCXR_reports.tgz"
EXTRACT_DIR = "NLMCXR_reports_extracted"
XML_FOLDER = os.path.join(EXTRACT_DIR, "ecgen-radiology")
OUTPUT_CSV = "converted_reports.csv"

def run_with_feedback(func, description):
    """Run a function with feedback messages."""
    print(f"\n🔄 {description}...")
    func()
    print(f"✅ {description} completed successfully!\n")

def clean_up():
    """Remove old files and directories if they exist."""
    if os.path.exists(TGZ_FILE):
        os.remove(TGZ_FILE)
    if os.path.exists(EXTRACT_DIR):
        shutil.rmtree(EXTRACT_DIR)
    if os.path.exists(OUTPUT_CSV):
        os.remove(OUTPUT_CSV)

def download_file():
    """Download the dataset file."""
    #print("🔄 Downloading dataset...")
    response = requests.get(TGZ_URL, stream=True)
    response.raise_for_status()
    with open(TGZ_FILE, "wb") as f:
        for chunk in response.iter_content(chunk_size=8192):
            f.write(chunk)
    print("✅ Dataset downloaded successfully!")

def extract_tgz():
    """Extract the dataset files."""
    #print("🔄 Extracting dataset files...")
    with tarfile.open(TGZ_FILE, "r:gz") as tar:
        tar.extractall(EXTRACT_DIR)
    print("✅ Files extracted successfully!")

def parse_xml_file(xml_path):
    """Parse a single XML file to extract findings and MeSH terms."""
    try:
        tree = ET.parse(xml_path)
        root = tree.getroot()

        # Extract AbstractText elements
        abstract_texts = root.findall(".//AbstractText")

        # Extract only the "Findings" section
        findings_text = ""
        for abstract in abstract_texts:
            label = abstract.attrib.get("Label", "").lower()
            if label == "findings":
                findings_text = abstract.text.strip() if abstract.text else ""
                break  # Stop after finding the "Findings" section

        # Extract MeSH terms
        mesh_major_list = [m.text.strip() for m in root.findall(".//MeSH/major") if m.text]
        mesh_major = "|".join(mesh_major_list) if mesh_major_list else ""

        return findings_text, mesh_major
    except:
        return "", ""

def download_prepare_dataset():
    """Handles dataset downloading, extraction, and conversion to CSV."""
    run_with_feedback(clean_up, "Cleaning up old files")

    run_with_feedback(download_file, "Downloading dataset")

    run_with_feedback(extract_tgz, "Extracting dataset")

    xml_files = glob.glob(os.path.join(XML_FOLDER, "*.xml"))

    print(f"\n🔄 Processing {len(xml_files)} XML files...")
    reports = [parse_xml_file(xml_file) for xml_file in tqdm(xml_files, desc="🔍 Parsing XML files")]
    print(f"✅ Processing completed! Total parsed reports: {len(reports)}\n")

    # Save reports to CSV
    print("🔄 Saving extracted data to CSV...")
    with open(OUTPUT_CSV, "w", newline="", encoding="utf-8") as csvfile:
        writer = csv.writer(csvfile)
        writer.writerow(["Report", "MeSH Major"])
        for report, mesh_major in reports:
            writer.writerow([report, mesh_major])
    print(f"✅ Data saved to {OUTPUT_CSV}!")

    # Load and clean the dataset
    df = pd.read_csv(OUTPUT_CSV)
    print(f"\n🔄 Cleaning dataset... {len(df)} entries found.")

    df["label"] = df["MeSH Major"].apply(lambda x: "normal" if x == "normal" else "abnormal")
    df.rename(columns={"Report": "report"}, inplace=True)
    df = df.map(lambda x: x.strip() if isinstance(x, str) else x)
    df = df.replace('', pd.NA).dropna().reset_index(drop=True)
    df.to_csv(OUTPUT_CSV, index=False)

    print("✅ Dataset preparation completed!")
    return df

# ✅ Run the full pipeline with feedback
df = download_prepare_dataset()

# Show data distribution
print(f'\n✅ Dataset ready! Total sample reports: {len(df)}')
print(df.label.value_counts())



🔄 Cleaning up old files...
✅ Cleaning up old files completed successfully!


🔄 Downloading dataset...
✅ Dataset downloaded successfully!
✅ Downloading dataset completed successfully!


🔄 Extracting dataset...
✅ Files extracted successfully!
✅ Extracting dataset completed successfully!


🔄 Processing 3955 XML files...


🔍 Parsing XML files: 100%|██████████| 3955/3955 [00:02<00:00, 1859.03it/s]


✅ Processing completed! Total parsed reports: 3955

🔄 Saving extracted data to CSV...
✅ Data saved to converted_reports.csv!

🔄 Cleaning dataset... 3955 entries found.
✅ Dataset preparation completed!

✅ Dataset ready! Total sample reports: 3425
label
abnormal    2219
normal      1206
Name: count, dtype: int64


Installing Ollama

In [16]:
import os
import subprocess
import threading
import time

MODEL_NAME = "llama3.1:8b"  # Change model name as needed

def run_command_with_feedback(command, description=None, check_existing=False):
    """Run a shell command with a description and print feedback."""
    if description is None:
        description = command
    print(f"\n🔄 {description}...")

    if check_existing:
        result = subprocess.run(command, shell=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)
        if result.returncode == 0:
            print(f"✅ {description} already completed, skipping.")
            return

    subprocess.run(command, shell=True, check=True)
    print(f"✅ {description} completed!")

def is_ollama_installed():
    """Check if Ollama is already installed."""
    result = subprocess.run("command -v ollama", shell=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
    return result.returncode == 0  # If the command returns 0, Ollama is installed

def is_ollama_running():
    """Check if Ollama is already running."""
    result = subprocess.run("pgrep -f 'ollama serve'", shell=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
    return result.returncode == 0  # If the command returns 0, Ollama is running

def is_model_downloaded(model_name):
    """Check if a model is already downloaded in Ollama."""
    result = subprocess.run(["ollama", "list"], stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)
    return model_name in result.stdout

def install_ollama():
    """Installs Ollama in Google Colab if not already installed."""
    if is_ollama_installed():
        print("✅ Ollama is already installed, skipping installation.")
        return

    print("\n🔄 Installing Ollama. This may take a few minutes...")
    run_command_with_feedback("apt update", "Updating package list")
    run_command_with_feedback("apt install -y pciutils", "Installing pciutils")
    run_command_with_feedback("curl -fsSL https://ollama.com/install.sh | sh", "Downloading and installing Ollama")
    print("✅ Ollama installed successfully!")

def start_ollama_server():
    """Starts the Ollama server in a background thread if not already running."""
    if is_ollama_running():
        print("✅ Ollama server is already running.")
        return

    print("\n🚀 Starting Ollama server...")
    thread = threading.Thread(target=lambda: subprocess.run(["ollama", "serve"], check=True))
    thread.start()
    time.sleep(5)  # Give it time to start
    if is_ollama_running():
        print("✅ Ollama server is running!")
    else:
        print("⚠️ Ollama server failed to start. Check for issues!")

def pull_model(model_name):
    """Downloads the model for Ollama if not already downloaded."""
    if is_model_downloaded(model_name):
        print(f"✅ Model '{model_name}' is already downloaded, skipping.")
        return

    print(f"\n🔄 Pulling model: {model_name}...")
    subprocess.run(["ollama", "pull", model_name], check=True)
    print(f"✅ Model '{model_name}' downloaded successfully!")

# ✅ Run the complete setup with verification
install_ollama()
start_ollama_server()
pull_model(MODEL_NAME)


✅ Ollama is already installed, skipping installation.
✅ Ollama server is already running.

🔄 Pulling model: llama3.1:8b...
✅ Model 'llama3.1:8b' downloaded successfully!


## **Classification with Large Language Models (LLMs)**
To facilitate classification, we define functions that analyze radiology reports using LLMs.

In [10]:
def classify_report(report, template, looking_for=None, examples=None):
    """
    Classify a radiology report using LLM
    """
    model=MODEL_NAME
    if 'gpt' in model:
        # Initialize OpenAI model. Remember that you need to provide an OpenAI API Key to run this model
        llm = ChatOpenAI(model="gpt-4o-mini-2024-07-18", temperature=0, seed=42, model_kwargs={"response_format": {"type": "json_object"}})
    else:
        # Initialize Ollama model
        llm = ChatOllama(model=model, base_url="localhost:11434", format='json', keep_alive=-1, temperature=0, seed=42, model_kwargs={"seed": 42, "response_format": {"type": "json_object"}})


    # Format the template with the report and any findings
    if looking_for and examples:
        prompt = template.format(report=report, looking_for=looking_for, examples=examples)
    else:
        prompt = template.format(report=report)

    try:
        response = llm.invoke(prompt)
        result = json.loads(response.content.lower())
        return result['classification']
    except Exception as e:
        print(f"Error during classification: {e}")
        return 'error'

def classify_multiple_reports(df, n_reports=5, template=None, looking_for=None, examples=None, seed=44):
    """Classify multiple random reports"""
    results = []
    random.seed(seed)  # Set seed for reproducibility
    report_indices = random.sample(range(len(df)), n_reports)

    for idx in tqdm(report_indices, desc="Classifying reports"):
        report = df.iloc[idx]['report']
        label = df.iloc[idx]['label']
        prediction = classify_report(report, template, looking_for, examples)

        results.append({
            'index': idx,
            'report': report,
            'original_label': label,
            'predicted_label': prediction
        })

    return results

def display_results(results, show_original_label=True):
    """Display classification results in a readable format"""

    for r in results:
        print(f"\nReport #{r['index']}:")
        print("=" * 50)
        print(f"Report text:")
        display(r['report'])
        print("-" * 50)

        if show_original_label:
            # Compare prediction with the original label
            is_correct = r['original_label'].lower() == r['predicted_label'].lower()
            classification = 'Correct' if is_correct else 'Wrong'
            classification_icon = '✅' if is_correct else '👎'
            print(f"Original Label : {r['original_label']}")
            print(f"Predicted Label: {r['predicted_label']}")
            print(f"Classification : {classification} {classification_icon}")

        else:
            # Only show predicted classification when original label is hidden
            is_correct = r['predicted_label'].lower() == "present"
            classification_icon = '✓✓✓' if is_correct else 'xxx'
            classification_icon = '✅' if is_correct else '👎'
            print(f"Finding: {r['predicted_label']} {classification_icon}")

        print()


    total = len(results)
    print("SUMMARY:")
    print("=" * 50)
    print(f"\nAnalyzed {total} reports")

    # Show accuracy if original labels are available
    if show_original_label:
        correct = sum(1 for r in results if r['original_label'].lower() == r['predicted_label'].lower())

        print(f"Correct predictions: {correct}")
        print(f"Accuracy: {(correct/total)*100:.1f}%\n")


# **Let's Try It Out!** 🏥

## **Zero-Shot Classification**
In **zero-shot classification**, the model determines whether a given chest X-ray report describes a normal or abnormal case without prior examples.


### How It Works:
**Prompt Template:**  
- The model receives a structured instruction to classify reports.
- The response format is **JSON**, containing the key **"classification"** with possible values: `"normal"` or `"abnormal"`.
- The `{report}` placeholder is replaced with an actual chest X-ray report before passing it to the model.


In [11]:
zero_shot_prompt_template = """
### INSTRUCTION
You are a specialist in chest X-ray reports. Your task is to classify whether a report is normal or abnormal.
Your response should be in JSON format with the key "classification" and the possible values: "normal" or "abnormal".

### REPORT TO CLASSIFY
{report}
"""


#for seed in range(60, 80):
results = classify_multiple_reports(df, n_reports=1, template=zero_shot_prompt_template, seed=73)
display_results(results, show_original_label=True)


Classifying reports: 100%|██████████| 1/1 [00:10<00:00, 10.16s/it]


Report #1146:
Report text:


'The lungs and pleural spaces show no acute abnormality. Heart size and pulmonary vascularity within normal limits.'

--------------------------------------------------
Original Label : normal
Predicted Label: normal
Classification : Correct ✅

SUMMARY:

Analyzed 1 reports
Correct predictions: 1
Accuracy: 100.0%



### Testing Classification in Multiple Reports to measure performance

In [12]:
results = classify_multiple_reports(df, n_reports=10, template=zero_shot_prompt_template, seed=44)

display_results(results, show_original_label=True)

Classifying reports: 100%|██████████| 10/10 [00:04<00:00,  2.06it/s]


Report #1673:
Report text:


'The heart is normal in size and contour. The lungs are clear, without evidence of infiltrate. There is no pneumothorax or effusion.'

--------------------------------------------------
Original Label : normal
Predicted Label: normal
Classification : Correct ✅


Report #2130:
Report text:


'The cardiomediastinal silhouette is normal. Lungs are hyperexpanded but clear without evidence of effusion or infiltrate. There is a small right lower lobe calcified granuloma that is unchanged from prior examinations. No acute bony abnormality. No pneumothorax or pneumomediastinum.'

...

--------------------------------------------------
Original Label : abnormal
Predicted Label: normal
Classification : Wrong 👎


Report #1186:
Report text:


'Heart size and pulmonary vascularity appear within normal limits. The lungs are free of focal airspace disease. No pleural effusion or pneumothorax is seen. Degenerative changes are present in the spine.'

--------------------------------------------------
Original Label : abnormal
Predicted Label: normal
Classification : Wrong 👎


Report #119:
Report text:


'Normal heart size and mediastinal contours. No focal airspace consolidation. Chronic appearing left greater than right lung base scarring with possible small effusions. No pneumothorax. Visualized osseous structures are unremarkable in appearance.'

--------------------------------------------------
Original Label : abnormal
Predicted Label: normal
Classification : Wrong 👎

SUMMARY:

Analyzed 10 reports
Correct predictions: 6
Accuracy: 60.0%



## 2. Few-Shot Classification

Zero-shot classification may not always be accurate. To enhance performance, we use **few-shot prompting**, where the model is provided with a few labeled examples to improve its accuracy.

### How It Works:
**Prompt Template:**  
- The `{examples}` placeholder contains relevant instances of abnormalities.
- The `{report}` placeholder is replaced with an actual chest X-ray report.
- The model is instructed to classify a report as **abnormal** if it contains specific **findings**.

In [13]:
few_show_template = """
### INSTRUCTION
You are a specialist in chest X-ray reports.
Your task is to classify a report as abnormal if it describes signs of {looking_for}, such as:
{examples}
Consider the finding positive even if it is mild.
If multiple X-rays are reported together, focus only on the chest X-ray report.
Your response should be in JSON format with the key 'classification' and the possible values: 'normal' or 'abnormal'.

### REPORT TO CLASSIFY
{report}
"""

looking_for = 'any abnormalities'

examples = """
- Low or high lung volume.
- Abnormalities in the lungs, bones, heart, mediastinum, or pleura.
- Abnormal calcifications, granulomas, or calcified lymph nodes.
- Post-surgical changes overlying the axilla, neck, or abdominal regions.
- Presence of surgical or closure devices.
- Part of the lung was not evaluated.
"""


print(f"\n{looking_for.capitalize()} Classifier")
print(f"Examples: {examples}")


results = classify_multiple_reports(df, n_reports=10,
                                    template=few_show_template,
                                    looking_for=looking_for,
                                    examples=examples,
                                    seed=44)

display_results(results, show_original_label=True)


Any abnormalities Classifier
Examples: 
- Low or high lung volume.
- Abnormalities in the lungs, bones, heart, mediastinum, or pleura.
- Abnormal calcifications, granulomas, or calcified lymph nodes.
- Post-surgical changes overlying the axilla, neck, or abdominal regions.
- Presence of surgical or closure devices.
- Part of the lung was not evaluated.



Classifying reports: 100%|██████████| 10/10 [00:05<00:00,  1.81it/s]


Report #1673:
Report text:


'The heart is normal in size and contour. The lungs are clear, without evidence of infiltrate. There is no pneumothorax or effusion.'

--------------------------------------------------
Original Label : normal
Predicted Label: normal
Classification : Correct ✅


Report #2130:
Report text:


...

--------------------------------------------------
Original Label : abnormal
Predicted Label: normal
Classification : Wrong 👎


Report #1186:
Report text:


'Heart size and pulmonary vascularity appear within normal limits. The lungs are free of focal airspace disease. No pleural effusion or pneumothorax is seen. Degenerative changes are present in the spine.'

--------------------------------------------------
Original Label : abnormal
Predicted Label: normal
Classification : Wrong 👎


Report #119:
Report text:


'Normal heart size and mediastinal contours. No focal airspace consolidation. Chronic appearing left greater than right lung base scarring with possible small effusions. No pneumothorax. Visualized osseous structures are unremarkable in appearance.'

--------------------------------------------------
Original Label : abnormal
Predicted Label: abnormal
Classification : Correct ✅

SUMMARY:

Analyzed 10 reports
Correct predictions: 8
Accuracy: 80.0%



## 3. Condition-Specific Classification

In some cases, you may need to classify reports based on specific medical conditions rather than general abnormalities.

### How It Works:
**Prompt Template:**  
- The `{looking_for}` placeholder specifies the condition to be detected (e.g., cardiomegaly, COPD).
- The `{examples}` placeholder contains key indicators related to the condition.
- The model classifies the report as **"present"** or **"absent"** based on the findings.

### Example: Cardiomegaly Detection
- **Signs of Cardiomegaly:**
  - Increased heart size
  - Enlarged cardiac silhouette
  - Increased cardiomediastinal silhouette

In [14]:
finding_specific_template = """
### INSTRUCTION
You are a specialist in chest X-ray reports.
Your task is to classify a report as positive if it describes signs of {looking_for}, such as:
{examples}
Consider the finding positive even if it is mild.
Your response should be in JSON format with the key 'classification' and the possible values: 'present' or 'absent'.

### REPORT TO CLASSIFY
{report}
"""

looking_for = 'cardiomegaly'

examples = """
- Increased heart size.
- Enlarged cardiac silhouette.
- Increased cardiomediastinal silhouette.
"""

print(f"\n{looking_for.capitalize()} Detection")
print(f"Examples: {examples}")


results = classify_multiple_reports(df,
                                    n_reports=10,
                                    template=finding_specific_template,
                                    looking_for=looking_for,
                                    examples=examples,
                                    seed=42)

display_results(results, show_original_label=False)


Cardiomegaly Detection
Examples: 
- Increased heart size.
- Enlarged cardiac silhouette.
- Increased cardiomediastinal silhouette.



Classifying reports: 100%|██████████| 10/10 [00:05<00:00,  1.86it/s]


Report #2619:
Report text:


'Mediastinal contours are normal. Unchanged XXXX opacity in the left lung base, XXXX scarring. Lungs are clear. There is no pneumothorax or large pleural effusion.'

--------------------------------------------------
Finding: absent 👎


Report #456:
Report text:


...

--------------------------------------------------
Finding: present ✅


Report #3016:
Report text:


'The heart is normal in size. The mediastinum is unremarkable. The lungs are clear.'

--------------------------------------------------
Finding: absent 👎


Report #419:
Report text:


'There is XXXX airspace disease in the right lower lobe seen behind the right hemidiaphragm on PA view. This is also well seen on lateral view. Remainder of the lungs appear clear. The heart and pulmonary XXXX appear normal. Mediastinal contours are normal.'

--------------------------------------------------
Finding: absent 👎

SUMMARY:

Analyzed 10 reports


# Create Your Own Specialized Classifier! 🚀

You can create a custom classifier for any specific condition by following these steps:

1. **Define the condition** you want to detect (e.g., COPD, pleural effusion).
2. **Specify key indicators** associated with the condition.
3. **Modify the template** to reflect the new condition and findings.
4. **Run the model** to classify reports based on your selected condition.

### Example: COPD Detection
- **Signs of COPD:**
  - Hyperinflated lungs
  - Flattened diaphragm
  - Increased retrosternal airspace
  - Narrowed cardiac silhouette
  - Emphysematous changes


In [ ]:
finding_specific_template = """
### INSTRUCTION
You are a specialist in chest X-ray reports.
Your task is to classify a report as positive if it describes signs of {looking_for}, such as:
{examples}
Consider the finding positive even if it is mild.
Your response should be in JSON format with the key 'classification' and the possible values: 'present' or 'absent'.

### REPORT TO CLASSIFY
{report}
"""

looking_for = "COPD (Chronic Obstructive Pulmonary Disease)"

examples = """
- Hyperinflated lungs.
- Flattened diaphragm.
- Increased retrosternal airspace.
- Narrowed cardiac silhouette.
- Emphysematous changes.
"""

print(f"\n{looking_for.capitalize()} Detection")
print(f"Examples: {examples}")


results = classify_multiple_reports(df,
                                    n_reports=10,
                                    template=finding_specific_template,
                                    looking_for=looking_for,
                                    examples=examples,
                                    seed=42)

display_results(results, show_original_label=False)

# Conclusion & Next Steps 📌

Through this notebook, we've explored how **prompt engineering** enhances the classification of radiology reports using LLMs.

### Key Takeaways:
✅ **Zero-shot prompting** enables classification without prior examples.  
✅ **Few-shot prompting** improves accuracy by providing labeled examples.  
✅ **Condition-specific classification** allows targeted detection of medical conditions.  

### What’s Next?
Now that you understand the fundamentals, here are some next steps:
1. **Test additional conditions** by modifying the `{looking_for}` and `{examples}` variables.
2. **Experiment with different LLM models** to compare performance.
3. **Refine your prompts** to improve classification accuracy.
4. **Apply real-world datasets** to validate the model in clinical settings.

By leveraging **LLMs and prompt engineering**, we can enhance efficiency in **radiology report classification**, aiding medical professionals in making informed decisions. 🏥💡

---

### Acknowledgment  
This project utilizes an **open-access chest X-ray collection** from **Indiana University**, sourced from the **OpenI repository** ([OpenI](https://openi.nlm.nih.gov/)). We acknowledge and appreciate the efforts of the dataset creators in making this valuable resource available for research and analysis.

---

### Credits
This notebook is adapted from work by **Paulo Kuriki** and **Felipe Kitamura**.

